In [47]:
import numpy as np
import glob
import csv
from rapidfuzz.string_metric import normalized_levenshtein
import json

In [49]:
def pair_from_row(row):
    pair={}
    pair["mass"] = float(row[1])
    pair["epsilon"] = float(row[2])
    pair["sigma"] = float(row[3])
    pair["m"] = float(row[4])
    pair["cut"] = float(row[5])
    pair["charge"] = 0.0
    return pair

def pair_of_h():
    pair={}
    pair["mass"] = 1.0
    pair["epsilon"] = 0.0
    pair["sigma"] = 1.0
    pair["m"] = 0
    pair["cut"] = 0.0
    pair["charge"] = 0.0
    return pair

def get_charge(row,pair_dict):
    keys = list(pair_dict.keys())
    dummy = np.zeros(len(keys))
    for i,key in enumerate(keys):
        dummy[i] = normalized_levenshtein(row[0][1:],key)
        #print(i, dummy[i])
    pointer = np.squeeze(np.where( dummy == np.amax(dummy)))
    charge = float(row[2])
    return keys[pointer],charge
    
def read_pair_potentials(path):
    keylist = ["!","#","models:","types","References"]
    pair_dict = {}
    flag=-1
    with open(path) as csvfile:
        spamreader = csv.reader(csvfile,delimiter=" ")
        for row in spamreader:
            row = [ x for x in row if x]
            if row:
                if "VdW-site" in row:
                    flag=1
                elif "coulomb-site" in row:
                    flag=2
                elif row[0] in keylist:
                    break            
                elif flag == 1 and row:
                    pair_dict[row[0]] = pair_from_row(row)
                elif flag == 2 and row:
                    print(row)
                    if row[0].split("_")[0] == "cH":
                        pair_dict[row[0]] = pair_of_h()
                        pair_dict[row[0]]["charge"] = float(row[2])
                    else:
                        p,ch = get_charge(row,pair_dict) 
                        print(p,ch)
                        pair_dict[p]["charge"] = ch
    return pair_dict

def read_json(path):
    with open(path) as json_file:
        return json.load(json_file)

In [50]:
path = glob.glob("*/pair_potentials")[0]
print(path)
print(read_pair_potentials(path))

UA_devel/pair_potentials
['cH_alcohol', '0.0', '0.404']
['cO_alcohol', '0.0', '-0.65']
OH_alcohol -0.65
['cC_alcohol', '0.0', '0.246']
CH2_alcohol 0.246
['cH_EOH', '0.0000', '+0.415']
['cO_EOH', '0.0000', '-0.667']
OH_EOH -0.667
['cC_EOH', '0.0000', '+0.252']
CH2_EOH 0.252
{'OH_alcohol': {'mass': 17.007, 'epsilon': 84.23, 'sigma': 3.035, 'm': 12.0, 'cut': 14.0, 'charge': -0.65}, 'CH2_alkane': {'mass': 14.027, 'epsilon': 52.9133, 'sigma': 4.04, 'm': 14.0, 'cut': 14.0, 'charge': 0.0}, 'CH2_alcohol': {'mass': 14.027, 'epsilon': 84.23, 'sigma': 3.842, 'm': 14.0, 'cut': 14.0, 'charge': 0.246}, 'CH2_EOH': {'mass': 14.027, 'epsilon': 76.265, 'sigma': 3.908, 'm': 14.0, 'cut': 14.0, 'charge': 0.252}, 'OH_EOH': {'mass': 17.007, 'epsilon': 76.265, 'sigma': 3.087, 'm': 12.0, 'cut': 14.0, 'charge': -0.667}, 'cH_alcohol': {'mass': 1.0, 'epsilon': 0.0, 'sigma': 1.0, 'm': 0, 'cut': 0.0, 'charge': 0.404}, 'cH_EOH': {'mass': 1.0, 'epsilon': 0.0, 'sigma': 1.0, 'm': 0, 'cut': 0.0, 'charge': 0.415}}


In [69]:
class molecule():    
    def __init__(self,mol):
        self.molecule = np.array(mol)
        self.f = np.zeros(len(mol))
        self.n = -1*np.ones(len(mol))
        self.i = np.arange(len(mol))
        n=0
        for i,m in enumerate(mol):
            if m[0]=="b":
                self.f[i] = int(m[1:])
            elif m[0]=="r":
                self.f[i] = -int(m[1:])
            else:
                self.n[i] = n
                n+=1
        return
    def get_distance(self,nos,ignore_ring=False):
        n_min = np.min(nos)
        n_max = np.max(nos)
        i_min = np.squeeze(np.where(self.n==n_min))
        i_max = np.squeeze(np.where(self.n==n_max))
        print(self.molecule[i_min],self.molecule[i_max])
        print(self.molecule,i_min,i_max,n_min,n_max,nos)
        
        if i_min==0:
            seq=[]
            i_lastmol = i_min
            lastaction = 0
            flag=0
            #print(np.arange(i_min,i_max))
            for i in np.arange(i_min,i_max+1):
                if self.f[i]==0:
                    if flag==0:
                        seq.append(self.molecule[i])
                        i_lastmol = i
                        lastaction = 0
                    else:
                        flag-=1
                elif flag==0:
                    n_end = self.n[i_lastmol]+self.f[i]+lastaction
                    i_end = np.squeeze(np.where(self.n==n_end))

                    if i_end < i_max and self.f[i]>0:
                        flag = self.f[i] 
                        lastaction += self.f[i]
                    elif self.f[i]<0: #ring closing...here??
                        print("aaah")
                    print(i,flag,i_end,i_max)
        else:
            print(i_min,i_max,len(self.n))
            l1,s1=self.get_distance((0,int(n_min) ))   
            l2,s2=self.get_distance((0,int(n_max) ))   
            last=0
            for i,s in enumerate(s1):
                if s in s2:
                    last=i
                else:
                    break
            p1 = np.squeeze(np.where(s1==s1[last]))
            p2 = np.squeeze(np.where(s2==s2[last]))
            print(last)
            print("test",s1,s2)
            print("test",p1,p2)
            print(s1[p1:])
            print(s2[p2:])
            ss1 = np.flip(s1[p1+1:])
            ss2 = s2[(p2):]
            seq= np.concatenate( (ss1,ss2) )
            
            
        return len(seq)-1,np.array(seq)
    
    def assign_xyz_coos(self,path):
        data = []
        with open(path) as csvfile:
            spamreader = csv.reader(csvfile,delimiter=" ")
            for row in spamreader:
                data.append([r for r in row if r])
        print(data)
        n = int(data[0][0])
        xyz = {}
        for i,d in enumerate(data[2:n+2]):
            p = self.molecule[np.squeeze(np.where(self.n==i))]
            if d[0] in p.split("_")[0]:
                xyz[i] = {}
                xyz[i]["atom"] = p
                xyz[i]["original"] = p
                xyz[i]["xyz"] = np.array([float(x) for x in d[1:4]])    
            else:
                print("ERROR: xyz doesnt fit your structure")
                break
        return xyz
    
    def assign_pair_potentials(self,path,args=["mass","epsilon","sigma","m","cut","charge"]):        
        self.ff  = read_pair_potentials(path)
        #{'mass': 17.007, 'epsilon': 84.23, 'sigma': 3.035, 'm': 12.0, 'cut': 14.0, 'charge': -0.65}
        self.mol_ff = [{}]*len(self.n)
        self.vdw_groups = 0
        for i,(n,m) in enumerate(zip(self.n,self.molecule)):
            if n >= 0:
                self.mol_ff[i] = self.ff[m]
                if self.mol_ff[i]["epsilon"] > 0.0:
                    self.vdw_groups += 1
            else:
                self.mol_ff[i] = None        
        return

    def write_ff(self):
        return
    
    def assign_gc(self,path,fun=read_json ):
        self.gc = fun(path)
        print(self.gc)
        self.mol_gc = [{}]*len(self.n)
        for i,(n,m) in enumerate(zip(self.n,self.molecule)):
            if n >= 0:
                self.mol_gc[i] = self.gc[m]
            else:
                self.mol_gc[i] = None      
        return
    
    def get_saft_from_ff(self,phi_sigma=1.0,phi_epsilon=1.0):
                
        sigmas   = np.array([ x["sigma"] for x in self.mol_ff ])
        epsilons = np.array([ x["epsilon"] for x in self.mol_ff ])
        
        self.m_saft       = np.sum(self.mol_gc)
        self.sigma_saft   = ( np.sum(sigmas**3)/self.m_saft )**(1/3) / phi_sigma
        sigma_i      = np.outer(sigmas,np.ones(self.vdw_groups))
        sigma_j      = np.outer(np.ones(self.vdw_groups),sigmas)
        sigma_ij     = (sigma_i+sigma_j)/2
        epsilon_i    = np.outer(epsilons,np.ones(self.vdw_groups))
        epsilon_j    = np.outer(np.ones(self.vdw_groups),epsilons)
        epsilon_ij   = np.sqrt(epsilon_i*epsilon_j)
        self.epsilon_saft = np.sum(epsilon_ij*sigma_ij**3) / ( phi_epsilon * self.m_saft**2 * (self.sigma_saft*phi_sigma)**3)        
        
        return
    

In [70]:
but = np.array(["cH_alcohol","OH_alcohol","CH2_alcohol","CH2_alkane","CH2_alkane","CH2_alcohol","OH_alcohol","cH_alcohol"])
eth12 = np.array(["cH_alcohol","OH_alcohol","CH2_alcohol","CH2_alcohol","OH_alcohol","cH_alcohol"])
eth = molecule(eth12)
eth.assign_xyz_coos("test.xyz")
path = glob.glob("*/pair_potentials")[0]
eth.assign_pair_potentials(path)
eth.assign_gc("group_contributions.json")
eth.get_saft_from_ff()
print(eth.gc)


[['6'], [], ['H', '-0.0007508', '-0.5866637', '-2.2385073'], ['O', '-0.0063283', '-0.6365049', '-1.277139'], ['C', '0.012032852941176468', '0.753978094117647', '-0.8163303'], ['C', '-0.01203267058823529', '0.7539781000000001', '0.8163303'], ['O', '0.006329', '-0.6365049', '1.277139'], ['H', '0.000751', '-0.5866637', '2.2385073']]
['cH_alcohol', '0.0', '0.404']
['cO_alcohol', '0.0', '-0.65']
OH_alcohol -0.65
['cC_alcohol', '0.0', '0.246']
CH2_alcohol 0.246
['cH_EOH', '0.0000', '+0.415']
['cO_EOH', '0.0000', '-0.667']
OH_EOH -0.667
['cC_EOH', '0.0000', '+0.252']
CH2_EOH 0.252
{'CH3_alkane': 0.61198, 'CH2_amine': 0.45606, 'CH2_alkane': 0.45606, 'NH2_amine': 0.40558, 'CH_alcohol': 0.14304, 'OH_alcohol': 0.402, 'CH2_alcohol': 1111.402, 'cH_alcohol': 0.0}


ValueError: operands could not be broadcast together with shapes (6,4) (4,6) 

In [66]:
mol = np.array(["A0","A1","A2","b7","B1","B2","b2","C1","C2","B3","B4","B5","r7","A3","A4"])
mol = np.array(["A0","A1","A2","b5","B1","B2","b2","C1","C2","B3","b2","D1","D2","b2","E1","E2","b2","F1","F2","A3","A4","A5"])

m = molecule(mol)

print(m.molecule,len(m.molecule))
print(m.f,len(m.f))
print(m.n,len(m.n))
print(m.i,len(m.i))


#print(m.get_distance((1,3)))
print("#################")
print(m.get_distance((1,11 )))
print("#################")
print(m.get_distance((1,13 )))
print("#################")
print(m.get_distance((1,15 )))    
print(m.get_distance((11,15 )))    
print(m.get_distance((5,15 )))    

['A0' 'A1' 'A2' 'b5' 'B1' 'B2' 'b2' 'C1' 'C2' 'B3' 'b2' 'D1' 'D2' 'b2'
 'E1' 'E2' 'b2' 'F1' 'F2' 'A3' 'A4' 'A5'] 22
[0. 0. 0. 5. 0. 0. 2. 0. 0. 0. 2. 0. 0. 2. 0. 0. 2. 0. 0. 0. 0. 0.] 22
[ 0.  1.  2. -1.  3.  4. -1.  5.  6.  7. -1.  8.  9. -1. 10. 11. -1. 12.
 13. 14. 15. 16.] 22
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] 22
#################
A1 E2
['A0' 'A1' 'A2' 'b5' 'B1' 'B2' 'b2' 'C1' 'C2' 'B3' 'b2' 'D1' 'D2' 'b2'
 'E1' 'E2' 'b2' 'F1' 'F2' 'A3' 'A4' 'A5'] 1 15 1 11 (1, 11)
1 15 22
A0 A1
['A0' 'A1' 'A2' 'b5' 'B1' 'B2' 'b2' 'C1' 'C2' 'B3' 'b2' 'D1' 'D2' 'b2'
 'E1' 'E2' 'b2' 'F1' 'F2' 'A3' 'A4' 'A5'] 0 1 0 1 (0, 1)
A0 E2
['A0' 'A1' 'A2' 'b5' 'B1' 'B2' 'b2' 'C1' 'C2' 'B3' 'b2' 'D1' 'D2' 'b2'
 'E1' 'E2' 'b2' 'F1' 'F2' 'A3' 'A4' 'A5'] 0 15 0 11 (0, 11)
3 5.0 9 15
10 2.0 12 15
13 0.0 15 15
1
test ['A0' 'A1'] ['A0' 'A1' 'A2' 'E1' 'E2']
test 1 1
['A1']
['A1' 'A2' 'E1' 'E2']
(3, array(['A1', 'A2', 'E1', 'E2'], dtype='<U2'))
#################
A1 F2
['A0' 'A1' 'A2' 'b5

In [7]:
np.squeeze(np.where(m.n==4))
np.flip(np.arange(1,5))

array([4, 3, 2, 1])

In [8]:

    
def read_xyz(path,energy=False):
    data = []
    with open(path) as csvfile:
        spamreader = csv.reader(csvfile,delimiter=" ")
        for row in spamreader:
            data.append([r for r in row if r])
    n = int(data[0][0])
    xyz = {}
    for i,d in enumerate(data[2:n+2]):
        xyz[i] = {}
        xyz[i]["atom"] = d[0]
        xyz[i]["xyz"] = np.array([float(x) for x in d[1:4]])          

    if energy:
        energy = float(data[1][2])
        return xyz,energy
        
    return xyz

def assign_CHx(xyz):
    xyz_CHx = {}
    xkeys = sorted(xyz.keys())
    ii=0
    x=np.min(xkeys)
    flag=0
    while x <= np.max(xkeys):
        if flag==0 and xyz[x]["atom"] == "C":
            coos = np.array(xyz[x]["xyz"])
            flag=1
            x+=1
        elif flag>0:
            if xyz[x]["atom"] == "H":
                coos = np.column_stack((coos, xyz[x]["xyz"]))
                x+=1
                flag+=1
            else:
                no=flag-1
                xyz_CHx[ii] = {}
                xyz_CHx[ii]["atom"] = "CH"+str(no)
                xyz_CHx[ii]["xyz"] = np.sum(coos*np.array([15]+[1]*no),axis=1)/(15+1*no)
                print(xyz_CHx[ii]["xyz"])
                coos = []
                flag=0
                ii+=1
        else:
            xyz_CHx[ii] = xyz[x]
            x+=1
            ii+=1
    return xyz_CHx
        
def to_xyz(xyz,path):
    f = open(path, "w")
    f.write(str(len(xyz))+"\n")
    f.write("\n")
    for x in xyz:
        line = "    ".join([xyz[x]["atom"][0]]+[str(y) for y in xyz[x]["xyz"]])
        f.write(line+"\n")
    f.close()
    

In [9]:
path = "tors_2.00.xyz"
xyz,ee =read_xyz(path,energy=True)
xxyz = assign_CHx(xyz)
print(len(xxyz))
for xx in xxyz:
    print(xxyz[xx])
    
to_xyz(xxyz,"test.xyz")
print(ee)

[ 0.01203285  0.75397809 -0.8163303 ]
[-0.01203267  0.7539781   0.8163303 ]
6
{'atom': 'H', 'xyz': array([-7.5080000e-04, -5.8666370e-01, -2.2385073e+00])}
{'atom': 'O', 'xyz': array([-0.0063283, -0.6365049, -1.277139 ])}
{'atom': 'CH2', 'xyz': array([ 0.01203285,  0.75397809, -0.8163303 ])}
{'atom': 'CH2', 'xyz': array([-0.01203267,  0.7539781 ,  0.8163303 ])}
{'atom': 'O', 'xyz': array([ 0.006329 , -0.6365049,  1.277139 ])}
{'atom': 'H', 'xyz': array([ 7.5100000e-04, -5.8666370e-01,  2.2385073e+00])}
-230.2123893649


In [10]:
[15]+[1]*3

[15, 1, 1, 1]

In [11]:
def read_bond_potentials(path):
    keylist = ["!","#","models:","types"]
    bond_dict = {}
    flag=0
    with open(path) as csvfile:
        spamreader = csv.reader(csvfile,delimiter=" ")
        for row in spamreader:
            row = [ x for x in row if x]
            if row:
                if row[0] == "model":
                    flag=1
                elif flag==1 and row[0] in keylist:
                    break
                elif flag==1:
                    name = "_".join(row[1:3])
                    bond_dict[name] = {}
                    bond_dict[name]["list"] = row[1:3]
                    bond_dict[name]["type"] = int(row[0]) 
                    #bond_dict[name]["len"] = float(row[3]) 
                    #bond_dict[name]["spring"] = float(row[4]) 
                    bond_dict[name]["p"] = [float(r) for r in row[3:]] 
                
    return bond_dict
                

In [12]:
path = glob.glob("*/bond_potentials")[0]
print(path)
print(read_bond_potentials(path))

UA_devel/bond_potentials
{'CH2_alkane_CH2_alkane': {'list': ['CH2_alkane', 'CH2_alkane'], 'type': 1, 'p': [1.54, 0.0]}, 'CH2_alcohol_CH2_alcohol': {'list': ['CH2_alcohol', 'CH2_alcohol'], 'type': 1, 'p': [1.514, 0.0]}, 'CH2_EOH_CH2_EOH': {'list': ['CH2_EOH', 'CH2_EOH'], 'type': 1, 'p': [1.514, 0.0]}, 'OH_EOH_cH_EOH': {'list': ['OH_EOH', 'cH_EOH'], 'type': 1, 'p': [0.97, 0.0]}, 'CH2_EOH_OH_EOH': {'list': ['CH2_EOH', 'OH_EOH'], 'type': 1, 'p': [1.42, 0.0]}, 'OH_alcohol_cH_alcohol': {'list': ['OH_alcohol', 'cH_alcohol'], 'type': 1, 'p': [0.97, 0.0]}, 'CH2_alcohol_OH_alcohol': {'list': ['CH2_alcohol', 'OH_alcohol'], 'type': 1, 'p': [1.42, 0.0]}, 'CH2_alkane_CH2_alcohol': {'list': ['CH2_alkane', 'CH2_alcohol'], 'type': 1, 'p': [1.514, 0.0]}}


In [13]:
def read_angle_potentials(path):
    keylist = ["!","#","models:","types"]
    angle_dict = {}
    flag=0
    with open(path) as csvfile:
        spamreader = csv.reader(csvfile,delimiter=" ")
        for row in spamreader:
            row = [ x for x in row if x]
            if row:
                if row[0] == "model":
                    flag=1
                elif flag==1 and row[0] in keylist:
                    break
                elif flag==1:
                    name = "_".join(row[1:4])
                    angle_dict[name] = {}
                    angle_dict[name]["list"] = row[1:4]
                    angle_dict[name]["type"] = int(row[0]) 
                    #angle_dict[name]["angle"] = float(row[4]) 
                    #angle_dict[name]["p"] = float(row[5]) 
                    angle_dict[name]["p"] = [float(r) for r in row[4:]] 
                
    return angle_dict

In [14]:
path = glob.glob("*/angle_potentials")[0]
print(path)
print(read_angle_potentials(path))

UA_devel/angle_potentials
{'CHxx_alkane_CH2_alkane_CHxx_alkane': {'list': ['CHxx_alkane', 'CH2_alkane', 'CHxx_alkane'], 'type': 2, 'p': [114.0, 62500.0]}, 'CH2_alcohol_OH_alcohol_cH_alcohol': {'list': ['CH2_alcohol', 'OH_alcohol', 'cH_alcohol'], 'type': 2, 'p': [107.4, 45960.0]}, 'CH2_alkane_CH2_alcohol_OH_alcohol': {'list': ['CH2_alkane', 'CH2_alcohol', 'OH_alcohol'], 'type': 2, 'p': [113.5, 62250.0]}, 'CH2_alcohol_CH2_alcohol_OH_alcohol': {'list': ['CH2_alcohol', 'CH2_alcohol', 'OH_alcohol'], 'type': 2, 'p': [113.5, 62250.0]}, 'CHxx_alkane_CH2_alkane_CH2_alcohol': {'list': ['CHxx_alkane', 'CH2_alkane', 'CH2_alcohol'], 'type': 2, 'p': [114.0, 62500.0]}, 'CH2_alcohol_CH2_alkane_CH2_alcohol': {'list': ['CH2_alcohol', 'CH2_alkane', 'CH2_alcohol'], 'type': 2, 'p': [114.0, 62500.0]}, 'CH2_EOH_OH_EOH_cH_EOH': {'list': ['CH2_EOH', 'OH_EOH', 'cH_EOH'], 'type': 2, 'p': [107.4, 45960.0]}, 'CH2_EOH_CH2_EOH_OH_EOH': {'list': ['CH2_EOH', 'CH2_EOH', 'OH_EOH'], 'type': 2, 'p': [113.5, 62250.0]}}


In [15]:
def read_torsion_potentials(path):
    keylist = ["!","#","models:","types","Note:"]
    torsion_dict = {}
    flag=0
    with open(path) as csvfile:
        spamreader = csv.reader(csvfile,delimiter=" ")
        for row in spamreader:
            row = [ x for x in row if x]
            if row:
                if row[0] == "model":
                    flag=1
                elif flag==1 and row[0] in keylist:
                    break
                elif flag==1:
                    name = "_".join(row[1:5])
                    torsion_dict[name] = {}
                    torsion_dict[name]["list"] = row[1:5]
                    torsion_dict[name]["type"] = int(row[0]) 
                    torsion_dict[name]["p"] = [float(r) for r in row[5:]] 
                
    return torsion_dict

In [16]:
path = glob.glob("*/torsion_potentials")[0]
print(path)
print(read_torsion_potentials(path))

UA_devel/torsion_potentials
{'CHxx_alkane_CH2_alkane_CH2_alkane_CHxx_alkane': {'list': ['CHxx_alkane', 'CH2_alkane', 'CH2_alkane', 'CHxx_alkane'], 'type': 1, 'p': [0.0, 355.03, -68.19, 791.32, 0.0]}, 'CH2_alcohol_CH2_alcohol_OH_alcohol_cH_alcohol': {'list': ['CH2_alcohol', 'CH2_alcohol', 'OH_alcohol', 'cH_alcohol'], 'type': 1, 'p': [-184.99, 82.0, 36.89, 303.85, 0.0]}, 'CH2_alkane_CH2_alcohol_OH_alcohol_cH_alcohol': {'list': ['CH2_alkane', 'CH2_alcohol', 'OH_alcohol', 'cH_alcohol'], 'type': 1, 'p': [-184.99, 82.0, 36.89, 303.85, 0.0]}, 'CHxx_alkane_CH2_alkane_CH2_alcohol_OH_alcohol': {'list': ['CHxx_alkane', 'CH2_alkane', 'CH2_alcohol', 'OH_alcohol'], 'type': 1, 'p': [0.0, 206.45, -222.56, 1085.08, 0.0]}, 'CH2_alcohol_CH2_alkane_CH2_alcohol_OH_alcohol': {'list': ['CH2_alcohol', 'CH2_alkane', 'CH2_alcohol', 'OH_alcohol'], 'type': 1, 'p': [0.0, 206.45, -222.56, 1085.08, 0.0]}, 'CHxx_alkane_CH2_alkane_CH2_alkane_CH2_alcohol': {'list': ['CHxx_alkane', 'CH2_alkane', 'CH2_alkane', 'CH2_alcoh

In [17]:
from scipy.constants import epsilon_0,k,e
print(epsilon_0,k,e)
def mieq(x1,x2,p1,p2):
    # TO DO:
    # - add cut-off
    # - add shift
    
    # distances
    r = np.linalg.norm(x1-x2)
    print("r:",r)
    
    # combining rules
    eps = np.sqrt(p1["epsilon"]*p2["epsilon"])
    sig = (p1["sigma"]+p2["sigma"])/2
    m   = (p1["m"]+p2["m"])/2
    print("sigma:",sig)
    
    # mie-force
    c0 = m / (m-6) * (m/6)**( 6/ (m-6) )
    mf = c0 * eps * ((sig / r) ** m - (sig / r) ** 6) 
   
    # q-force
    qf  = (p1["charge"]*p2["charge"] *1e10*e**2)/(4*np.pi*epsilon_0*r*k)
    
    print(e * e * 1.0e10 / ( 4.0 * np.pi * epsilon_0 * k ))
    
    qf3 = p1["charge"]*p2["charge"]*167100.002729011/r
    
    return mf,qf,qf3

8.8541878128e-12 1.380649e-23 1.602176634e-19


In [25]:
path = glob.glob("*/pair_potentials")[0]
ff  = read_pair_potentials(path)
xyz = eth.assign_xyz_coos("test.xyz")
#print(ff)
mf,qf,qf2 = mieq(xyz[1]["xyz"],xyz[5]["xyz"],ff["OH_alcohol"],ff["CH2_alcohol"])
#print(mf,qf,qf2,mf-qf)
mf,qf,qf2 = mieq(xyz[1]["xyz"],xyz[4]["xyz"],ff["OH_alcohol"],ff["CH2_alcohol"])
#print(mf,qf,qf2,mf-qf)
mf,qf,qf2 = mieq(xyz[1]["xyz"],xyz[3]["xyz"],ff["OH_alcohol"],ff["CH2_alcohol"])
#print(mf,qf,qf2,mf-qf)
#x1=np.array([1,0,0])
#x2=np.array([1,1,0])
#np.linalg.norm(x1-x2)
print(ff)

['cH_alcohol', '0.0', '0.404']
['cO_alcohol', '0.0', '-0.65']
OH_alcohol -0.65
['cC_alcohol', '0.0', '0.246']
CH2_alcohol 0.246
['cH_EOH', '0.0000', '+0.415']
['cO_EOH', '0.0000', '-0.667']
OH_EOH -0.667
['cC_EOH', '0.0000', '+0.252']
CH2_EOH 0.252
[['6'], [], ['H', '-0.0007508', '-0.5866637', '-2.2385073'], ['O', '-0.0063283', '-0.6365049', '-1.277139'], ['C', '0.012032852941176468', '0.753978094117647', '-0.8163303'], ['C', '-0.01203267058823529', '0.7539781000000001', '0.8163303'], ['O', '0.006329', '-0.6365049', '1.277139'], ['H', '0.000751', '-0.5866637', '2.2385073']]
r: 3.516006707674151
sigma: 3.4385000000000003
167100.94689828737
r: 2.5543093603804707
sigma: 3.4385000000000003
167100.94689828737
r: 2.5131830858843727
sigma: 3.4385000000000003
167100.94689828737
{'OH_alcohol': {'mass': 17.007, 'epsilon': 84.23, 'sigma': 3.035, 'm': 12.0, 'cut': 14.0, 'charge': -0.65}, 'CH2_alkane': {'mass': 14.027, 'epsilon': 52.9133, 'sigma': 4.04, 'm': 14.0, 'cut': 14.0, 'charge': 0.0}, 'CH2_

In [26]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(x1, x2, x3):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1 = unit_vector(x1-x2)
    v2 = unit_vector(x3-x2)
    
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def bend(ang,p):

    if p["type"] == 2:
        force = p["p"][1]*(ang-p["p"][0]*np.pi/180)**2  
    else:
        force=0.0
    return force
    

In [20]:
path = glob.glob("*/angle_potentials")[0]

angs = read_angle_potentials(path)
#print(angs)

bend(angle_between(xyz[1]["xyz"],xyz[2]["xyz"],xyz[3]["xyz"]),angs["CHxx_alkane_CH2_alkane_CHxx_alkane"])
bend(angle_between(np.array([1,0,0]),np.array([0,0,0]),np.array([0,1,0])),angs["CHxx_alkane_CH2_alkane_CHxx_alkane"])

10966.227112321505

In [21]:
def dihedral(x0,x1,x2,x3):
    """Praxeolitic formula
    1 sqrt, 1 cross product"""

    b0 = -1.0*(x1 - x0)
    b1 = x2 - x1
    b2 = x3 - x2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    #print(b1)
    #print(np.linalg.norm(b1))
    if np.linalg.norm(b1)>0:
        b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.arctan2(y, x)
    return np.degrees(np.arctan2(y, x))

In [22]:
# some atom coordinates for testing
p0 = np.array([24.969, 13.428, 30.692]) # N
p1 = np.array([24.044, 12.661, 29.808]) # CA
p2 = np.array([22.785, 13.482, 29.543]) # C
p3 = np.array([21.951, 13.670, 30.431]) # O
p4 = np.array([23.672, 11.328, 30.466]) # CB
p5 = np.array([22.881, 10.326, 29.620]) # CG
p6 = np.array([23.691,  9.935, 28.389]) # CD1
p7 = np.array([22.557,  9.096, 30.459]) # CD2

dihedral(p0,p1,p2,p3)
dihedral(p1,p5,p2,p3)


-2.1231360602400833

In [23]:
def torsion(ang,p):
    
    print(ang)
    force = p[0] + p[1]* (1 + np.cos( ang )) + p[2]* (1 - np.cos(2* ang )) + p[3]* (1 + np.cos(3* ang ))
    
    return force

In [24]:
path = glob.glob("*/torsion_potentials")[0]

toto = read_torsion_potentials(path)
print(toto["CHxx_alkane_CH2_alkane_CH2_alkane_CHxx_alkane"]["p"])

print(torsion(dihedral(p1,p5,p2,p3),toto["CHxx_alkane_CH2_alkane_CH2_alkane_CHxx_alkane"]["p"]))
print(torsion(dihedral(p0,p1,p2,p3),toto["CHxx_alkane_CH2_alkane_CH2_alkane_CHxx_alkane"]["p"]))

[0.0, 355.03, -68.19, 791.32, 0.0]
-2.1231360602400833
1649.617019945443
-1.2429388648155737
479.67725417596534
